## DeepLesion: automated mining of large-scale lesion annotations and universal lesion detection with deep learning
### Ke Yan$^{a}$, Xiaosong Wang$^{a}$, Le Lu$^{b}$, Ronald M. Summers$^{a}$
#### $^{a}$ National Institutes of Health, Clinical Center, Imaging Biomarkers and Computer-Aided Diagnosis Laboratory, Bethesda, Maryland, United States
#### $^{b}$ National Institutes of Health, Clinical Center, Clinical Image Processing Service, Radiology and Imaging Sciences, Bethesda, Maryland, United States

 Journal of Medical Imaging
 
 Paper 18043R received Mar. 7, 2018; accepted for publication Jun. 14, 2018.
 
---

## 개요
현재 병변 감지(lesion detection)를 위해 존재하는 알고리즘들은 하나의 병변을 타겟으로 한다.
* 종양은 전이성을 지니고 있어 지역 림프절이나 다른 신체 부위로 퍼질 가능성 존재

![CT](./resource/image/multi_lesion.png)

## 논문에서의 시도
1. 최소한의 작업으로 PACS에 수집된 Bookmark에서 병변 주석을 수집하는 패러다임을 도입
2. 이 데이터 세트를 사용하여 하나의 통일된 프레임워크로 모든 유형의 병변의 위치를 찾을 수 있는 자동 병변 감지 알고리즘을 개발

## Bookmark
방사선 전문의는 영상 이미지를 보고 임상적 주석을 달고 측정한다.

![bookmark](./resource/bookmark.png)

>RECIST-diameter
>- 두개의 선으로 구성
>- 하나는 병변에서 가장 긴 직경을 측정
>- 다른 하나는 측정면에서 가장 긴 직경을 측정
>- 병변의 정확한 위치와 크기를 알수 있음

## Image Preprocessing

### CT 이미지 처리 절차
![img_preprocessing](./resource/image/img_preprocessing.png)

### Bookmark의 데이터화 
![bookmark](./resource/image/bookmark2csv.png)


### CSV파일 설명

|변수명|설명|값|비고|
|---|---|---|---|
|File_name|파일명|string||
|Patient_index|환자 Index|number||
|Study_index|연구 Index|number||
|Series_ID| Series ID|number||
|Key_slice_index|Slice Index|number||
|Measurement_coordinates| 병변의 RECIST 직경의 영상 좌표|[x11,y11, x12, y12, x21, y21, x22, y22]|<img src="./resource/Measurement_coordinates.png" width="300"/>|
|Bounding_boxes| RECIST 직경에서 추정된 병변의 경계| [x1, y1, x2, y2]|<img src="./resource/Bounding_boxes.png" width="300"/>|
|Lesion_diameters_Pixel_|긴 축과 짧은 축의 길이|[장축 길이, 단축 길이]|<img src="./resource/RECIST.png" width="300"/>|
|Normalized_lesion_location| 중앙의 상대적 신체 위치|[x,y,z]||
|Coarse_lesion_type|병변의 위치|1:뼈(Bone) 2:복부(Abdomen): 간이나 신장에 없는 복강의 병변 3:종격(Mediastinum) : 양쪽 폐 사이의 공간 4:간(Liver) 5:폐(Lung) 6:신장(Kidney) 7:연조직(Soft tiss) : 신체벽, 근육, 피부, 지방, 사지, 머리, 목의 잡다한 병변 8:골반(Pelvis)||
|Possibly_noisy| 수동 확인으로 병변에 노이즈가 있을 경우 1로 설정|0:이상없음 1:노이즈 존재||
|Slice_range|Slice 범위(Key slice와 인접한 slice 이미지 제공 범위)|Bookmark가 표기된 Key slice가 63일때 slice image는 57부터 69까지 제공되었다. [첫 slice index, 마지막 slice index]||
|Spacing_mm_px_| pixel당 mm거리| [x,y,z]||
|Image_size|이미지 크기|[x,y]||
|Patient_gender|환자 성별|M:남자, F:여자||
|Patient_age|환자 나이|number||
|Train_Val_Test|data split|1:train data 2:validation data 3:test data||


### CSV파일 Data Preview

In [1]:
import pandas as pd

csv_path = 'DL_info.csv'
info_df = pd.read_csv(csv_path)

info_df.sample(5)

,File_name,Patient_index,Study_index,Series_ID,Key_slice_index,Measurement_coordinates,Bounding_boxes,Lesion_diameters_Pixel_,Normalized_lesion_location,Coarse_lesion_type,Possibly_noisy,Slice_range,Spacing_mm_px_,Image_size,DICOM_windows,Patient_gender,Patient_age,Train_Val_Test
23,000089_11_01_036.png,89,11,1,36,"147.4, 212.253, 136.407, 235.811, 150.73, 228....","128.079, 207.253, 155.73, 240.811","25.9964, 19.4786","0.184358, 0.338804, 0.544956",-1,0,"30, 42","0.820312, 0.820312, 5","512, 512","-160, 240",F,74,1
35,000089_14_02_184.png,89,14,2,184,"131.018, 218.817, 131.018, 185.742, 122.599, 2...","117.599, 180.742, 148.045, 223.817","33.0749, 20.4463","0.215073, 0.357627, 0.54569",-1,0,"154, 214","0.679688, 0.679688, 1","512, 512","-175, 275",F,74,1
142,000098_08_01_021.png,98,8,1,21,"371.657, 167.813, 367.481, 153.008, 364.824, 1...","359.824, 148.008, 380.453, 172.813","15.3819, 11.0434","0.703827, 0.217028, 0.451728",7,0,"15, 27","0.781, 0.781, 5","512, 512","-175, 275",F,64,2
102,000096_04_01_030.png,96,4,1,30,"256.146, 280.712, 271.662, 289.116, 264.551, 2...","251.146, 275.066, 276.662, 294.116","17.645, 10.293","0.509042, 0.585147, 0.500964",-1,0,"24, 36","0.974609, 0.974609, 5","512, 512","-175, 275",M,72,1
69,000094_03_01_052.png,94,3,1,52,"297.375, 209.963, 270.569, 217.539, 285.72, 21...","265.569, 194.474, 302.375, 223.704","27.8562, 19.6585","0.558176, 0.469917, 0.570192",-1,0,"46, 58","0.976562, 0.976562, 5","512, 512","-175, 275",M,67,1


## 신경망분석
![froc1](./resource/image/analysis.png)

### 신경망 세부구조
![froc1](./resource/image/map.png)

## FROC를 통한 모형 예측 평가
### 병변 유형별 FROC
![froc1](./resource/image/froc1.png)

### 병변 크기별 FROC
![froc1](./resource/image/froc2.png)

## 해당 연구의 한계
* 겹쳐진 병변의 분할 
* 3차원 경계상자 미사용
* CT 이미지에는 대표적인 병변만 표시 되었으므로, 일부 CT이미지에는 병변이 있음에도 주석 되지 않음